In [1]:
pip install segmentation-models-pytorch

  Using cached safetensors-0.5.3-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
Using cached safetensors-0.5.3-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (471 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os

ARCHITECTURE = 'deeplabv3'
os.environ['ARCHITECTURE'] = ARCHITECTURE

In [2]:
from landnet.modelling.segmentation.models import (
    DeepLabV3ResNet50Builder,
    FCNResNet50Builder,
)
from pathlib import Path
from landnet.enums import GeomorphometricalVariable, Mode
from landnet.features.tiles import TileConfig, TileSize
from landnet.features.grids import get_grid_for_variable
import torch
from landnet.modelling.segmentation.lightning import (
    LandslideImageSegmenter,
    LandslideImageSegmentationDataModule,
)
from landnet.modelling.dataset import (
    get_default_mask_transform,
    get_default_transform,
    get_default_augment_transform,
)
from landnet.modelling.tune import MetricSorter
from landnet.modelling.segmentation.dataset import (
    ConcatLandslideImageSegmentation,
    LandslideImageSegmentation,
)
from landnet.modelling.segmentation.inference import Infer
from landnet.modelling import torch_clear
import lightning as L
from landnet.typing import TuneSpace
import typing as t

torch_clear()

/home/alex/miniforge3/envs/landnet/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
/home/alex/miniforge3/envs/landnet/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-20 16:46:46,519	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-06-20 16:46:46,775	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-06-20 16:46:46,879	INFO util.py:1

In [3]:
variables = [
    GeomorphometricalVariable.SLOPE,
    GeomorphometricalVariable.REAL_SURFACE_AREA,
    GeomorphometricalVariable.PROFILE_CURVATURE,
    # GeomorphometricalVariable.DIGITAL_ELEVATION_MODEL,
    # GeomorphometricalVariable.GENERAL_CURVATURE,
    # GeomorphometricalVariable.HILLSHADE,
    # GeomorphometricalVariable.TOPOGRAPHIC_POSITION_INDEX,
    # GeomorphometricalVariable.NEGATIVE_TOPOGRAPHIC_OPENNESS,
]
tile_config = TileConfig(TileSize(20, 20), overlap=0)
model_config: TuneSpace = {
    'batch_size': 4,
    'learning_rate': 0.00001,
    'tile_config': tile_config,
}

In [4]:
train_grids = [
    get_grid_for_variable(
        variable,
        tile_config=tile_config,
        mode=Mode.VALIDATION,
    )
    for variable in variables
]

validation_grids = [
    get_grid_for_variable(
        variable,
        tile_config=tile_config,
        mode=Mode.TRAIN,
    )
    for variable in variables
]

train_dataset = ConcatLandslideImageSegmentation(
    landslide_images=[
        LandslideImageSegmentation(
            grid,
            Mode.VALIDATION,
            transform=get_default_transform(),
            mask_transform=get_default_mask_transform(),
        )
        for grid in train_grids
    ],
    augment_transform=None,
)

validation_dataset = ConcatLandslideImageSegmentation(
    landslide_images=[
        LandslideImageSegmentation(
            grid,
            Mode.TRAIN,
            transform=get_default_transform(),
            mask_transform=get_default_mask_transform(),
        )
        for grid in validation_grids
    ],
    augment_transform=None,
)

# train_dataset, validation_dataset = torch.utils.data.random_split(
#     dataset, (0.7, 0.3)
# )
# t.cast(
#     ConcatLandslideImageSegmentation, train_dataset
# ).augment_transform = get_default_augment_transform()

test_grids = [
    get_grid_for_variable(
        variable,
        tile_config=tile_config,
        mode=Mode.TEST,
    )
    for variable in variables
]
test_dataset = ConcatLandslideImageSegmentation(
    landslide_images=[
        LandslideImageSegmentation(
            grid,
            Mode.TEST,
            transform=get_default_transform(),
            mask_transform=get_default_mask_transform(),
        )
        for grid in test_grids
    ],
    augment_transform=None,
)

/home/alex/miniforge3/envs/landnet/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [ ]:
model = DeepLabV3ResNet50Builder().build(
    in_channels=len(variables), mode=Mode.VALIDATION
)
dm = LandslideImageSegmentationDataModule(
    model_config,
    variables,
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    # validation_dataset=validation_dataset,
)
segmenter = LandslideImageSegmenter(model_config, model, 2)
trainer = L.Trainer(
    max_epochs=5, enable_checkpointing=True, limit_val_batches=0
)

INFO: Set weights to DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1


INFO: You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs


In [6]:
model = trainer.fit(model=segmenter, datamodule=dm)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name            | Type                         | Params | Mode 
-------------------------------------------------------------------------
0 | model           | DeepLabV3                    | 39.6 M | train
1 | criterion       | CrossEntropyLoss             | 0      | train
2 | train_metrics   | SegmentationMetricCollection | 0      | train
3 | val_metrics     | SegmentationMetricCollection | 0      | train
4 | test_metrics    | SegmentationMetricCollection | 0      | train
5 | predict_metrics | SegmentationMetricCollection | 0      | train
-------------------------------------------------------------------------
39.6 M    Trainable params
0         Non-trainable params
39.6 M    Total params
158.536   Total estimated model params size (MB)
200       Modules in train mode
0         Modules in eval mode


Epoch 4: 100%|██████████████████████████████████████████████████████| 250/250 [01:51<00:00,  2.24it/s, v_num=21, train_dice_score=0.976, train_generalized_dice_score=0.982, train_mIoU=0.488]

INFO: `Trainer.fit` stopped: `max_epochs=5` reached.



poch 4: 100%|██████████████████████████████████████████████████████| 250/250 [02:01<00:00,  2.05it/s, v_num=21, train_dice_score=0.976, train_generalized_dice_score=0.982, train_mIoU=0.488]

In [7]:
trainer.test(model=segmenter, dataloaders=dm)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0:  23%|█████████████████████████████▋                                                                                                  | 580/2500 [01:57<06:29,  4.93it/s]

INFO: 
Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
infer = Infer(variables, model_config)
parent = Path(
    '/media/alex/alex/python-modules-packages-utils/landnet/notebooks/lightning_logs'
)
last_version = sorted(
    map(lambda x: int(x.name.split('_')[-1]), parent.glob('version*'))
)[-1]
# last_version = '8'
ckpt = f'/media/alex/alex/python-modules-packages-utils/landnet/notebooks/lightning_logs/version_{last_version}/checkpoints/epoch=4-step=1250.ckpt'
infer.handle_checkpoint(ckpt)

INFO: You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



redicting DataLoader 0: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2500/2500 [08:09<00:00,  5.10it/s]

INFO: You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0:   2%|██                                                                                                                            | 40/2500 [00:05<05:18,  7.72it/s]